### T&uuml;rme von Hanoi mit Klassen  
Die Komponenten Game, View und Controller sind nun als Klassen implementiert.

- Die Klasse `Game` implementiert u.a. die Methoden
`new_game`, `move_disk` und `register_callback` und jede Instanz der Klasse Game hat die Attribute `stacks` und `_callbacks`.  
Das f&uuml;hrende `_` des Variabelnamen deutet an, dass `_callbacks` eine private Variable ist, und nur innerhalb der Klasse `Game` angeprochen werden sollte.

- Die Klasse `View` implementiert u.a. die Methode `draw_stacks`, welche den Spielzustand auf der Leinwand darstellt. Die Methode `_ipython_display_` wird von Jupyterlab ausgef&uuml;hrt wenn `display(view)` aufgerufen wird .

- Die Klasse `Controller` benutzt das Canvas-Widget einer `View`-Instanz um Tastendr&uuml;cke entgegenzunehmen, zu interpretieren und entsprechende Kommandos das Instanz-Attribut `game` zu schicken.
 
Eine Controller-Instanz `Controller(game, canvas)`
nutzt die Canvas der View um Tastaturinput entgegenzunehmen.
Je nach Input wird dann `game.new_game()` oder `game.move_disk(src, dst)` aufgerufen.
Nach getaner Arbeit rufen dann `new_game` bez. `move_disk` das von der View registrierte Callback auf, was zur Darstellung des Spielzustandes führt.

**To do**:  
Schreibe eine Hilfsmethode `is_legal(i, j)`, die `True` zur&uuml;ck gibt, falls die Verschiebung m&ouml;glich und regelkonform ist. `move_disk(i, j)` soll dann nur Etwas tun, falls `is_legal(i, j)` `True` liefert.

In [ ]:
#%%file game.py
class Game:
    def __init__(self):
        self.stacks = []
        self._callbacks = []
    
    def register_callback(self, callback):
        if callback not in self._callbacks:
            self._callbacks.append(callback)
            
    def new_game(self):
        self.stacks[:] = [list(range(4)), [], []]
        self._notify('new_game', self.stacks)
        
    def move_disk(self, i, j):   
        if self._is_legal(i, j):
            x = self.stacks[i].pop()
            self.stacks[j].append(x)
            self._notify('move_disk', self.stacks)
            
    def _is_legal(self, i,j):
        '''sollte testen, ob Verschiebung moeglich und regelkonform'''
        return True        
            
    def _notify(self, event, data):
        for f in self._callbacks:
            f(event, data)

In [ ]:
# a first callback
f = lambda event, data: print('Event: {}\nData: {}'.format(event, data))
f('test', 1)

In [ ]:
game = Game()

In [ ]:
# wirf mit <tab> einen BLick auf die oeffentilchen Attribute des Objektes game
game.

In [ ]:
game = Game()
game.register_callback(f)
game.new_game()

In [ ]:
game.move_disk(0,1)

In [ ]:
#%%file view.py
class View:
    from ipycanvas import Canvas
    
    positions = [50, 150, 250]
    disk_widths = [90, 70, 50, 30]
    disk_height = 10
    stack_width = 100
    colors = ['#8f5902', '#204a87', '#c4a000', '#f57900'] 
    
    def __init__(self, game):
        game.register_callback(self.callback)
        self.canvas = View.Canvas(width = 300, height = 100, layout={'border': '1px solid black'})
        
    def draw_stack(self, stack, x):
        self.canvas.clear_rect(x - self.stack_width/2, 0, self.stack_width, self.canvas.height)
        h = self.disk_height
        for i, disk in enumerate(stack):
            self.canvas.fill_style = self.colors[disk]
            w = self.disk_widths[disk]
            self.canvas.fill_rect(x - w/2, self.canvas.height-(i+1)*h, w, h)
        
    def draw_stacks(self, stacks):
        for x, stack in zip(self.positions, stacks):
            self.draw_stack(stack, x)    
     
    def callback(self, event, data):
        self.draw_stacks(data)
        
    def _ipython_display_(self):
        display(self.canvas)

In [ ]:
view = View(game)
view

In [ ]:
game.new_game()

In [ ]:
game.move_disk(0,2)

In [ ]:
game._callbacks[-1]

In [ ]:
# Speicheradresse in hex des Objektes view
hex(id(view))

In [ ]:
#%%file controller.py
class Controller:
    def __init__(self, game, canvas):
        canvas.on_key_down(self.on_key_down)
        self.game = game
        self.src = None

    def on_key_down(self, key, *flags):
        if key in '123' and self.src is None:
            self.src = int(key) - 1
        elif key in '123' and self.src is not None:
            dst = int(key) - 1
            self.game.move_disk(self.src, dst)
            self.src = None
        elif key == 'n':
            self.game.new_game()
            self.src = None    
        else:
            self.src = None

In [ ]:
# canvas der view hoert nun auf Tastendruecke
controller = Controller(game, view.canvas)

### Komponenten in separaten Files speichern
Speichere die Klassen `Game`, `View` und `Controller` in separaten Files
`game.py`, `view.py` und `controller.py`.  &Ouml;ffne ein neues Notebook.

Importiere dort die Klassen Game, View und Controller:
```python
from game import Game
from view import View
from controller import Controller

```
Starte die Applikation mit
```python
game = Game()
view = View(game)
controller = Controller(game, view.canvas)

display(view)
```